In [38]:
import cellCnn

importlib.reload(cellCnn)
import random
import numpy as np
### (from: https://github.com/eiriniar/CellCnn/blob/0413a9f49fe0831c8fe3280957fb341f9e028d2d/cellCnn/examples/NK_cell_ungated.ipynb ) AND https://github.com/eiriniar/CellCnn/blob/0413a9f49fe0831c8fe3280957fb341f9e028d2d/cellCnn/examples/PBMC.ipynb
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.utils import shuffle
import seaborn as sns
import matplotlib.pyplot as plt
from cellCnn.ms.utils.helpers import get_chunks
from cellCnn.ms.utils.helpers import print_regression_model_stats
from cellCnn.plotting import plot_results
from cellCnn.utils import mkdir_p
from cellCnn.utils import save_results
from cellCnn.ms.utils.helpers import get_fitted_model
from cellCnn.ms.utils.helpers import split_test_valid_train
from cellCnn.ms.utils.helpers import calc_frequencies, get_chunks_from_df
#% pylab inline

In [143]:
#%reset

In [144]:
##### state vars
cytokines = ['CCR2', 'CCR4', 'CCR6', 'CCR7', 'CXCR4', 'CXCR5', 'CD103', 'CD14', 'CD20', 'CD25', 'CD27', 'CD28', 'CD3',
             'CD4', 'CD45RA', 'CD45RO', 'CD56', 'CD57', 'CD69', 'CD8', 'TCRgd', 'PD.1', 'GM.CSF', 'IFN.g', 'IL.10',
             'IL.13', 'IL.17A', 'IL.2', 'IL.21', 'IL.22', 'IL.3', 'IL.4', 'IL.6', 'IL.9', 'TNF.a']
infile = 'cohort_denoised_clustered_diagnosis_patients.csv'
indir = 'data/input'
outdir = 'out_ms_default'
rand_seed = 123
train_perc = 0.7
test_perc = 0.3
batch_size_pheno = 840  # so a size of 8425 is about equally sized in batches
batch_size_cd4 = 550  # so a size of 550 gets me 16 batches for cd4
## information from ms_data project
cluster_to_celltype_dict = {0: 'b', 1: 'cd4', 3: 'nkt', 8: 'cd8', 10: 'nk', 11: 'my', 16: 'dg'}

In [145]:
np.random.seed(rand_seed)
mkdir_p(outdir)
df = pd.read_csv(os.path.join(indir, infile), index_col=0)
df = df.drop_duplicates()  ### reduces overfitting at cost of fewer data
df.shape
##### no duplicates in

(16889, 38)

In [146]:
plt.figure()
patients_clusters_conf_table = pd.crosstab(df['gate_source'], df['cluster'])
sns.heatmap(patients_clusters_conf_table, annot=False, vmax=100)
plt.show()
#plt.savefig('images/patient_vs_cluster_conf_table.png')
patients_clusters_conf_table

/var/folders/p1/5d38cf1d6tvdyny38zb_66540000gn/T/ipykernel_19846/2024332628.py:4: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


cluster,0,1,3,8,10,11,16
gate_source,,,,,,,
3.0,60,119,6,30,5,5,0
4.0,15,46,0,45,2,1,13
5.0,33,131,0,70,2,2,4
8.0,24,122,0,31,10,11,4
9.0,72,270,2,137,11,10,4
...,...,...,...,...,...,...,...
128.0,4,6,4,6,1,6,0
130.0,24,82,2,56,8,4,0
131.0,20,48,2,14,13,7,0


In [147]:
# pitch: key = gate_source, value = dict{diagnosis: df OR freq?}
rrms_df = df[df['diagnosis'] == 'RRMS']
rrms_patients2df = {id: patient_df.drop(columns=['diagnosis', 'gate_source']) for id, patient_df in
                    rrms_df.groupby('gate_source')}
nindc_df = df[df['diagnosis'] == 'NINDC']
nindc_patients2df = {id: patient_df.drop(columns=['diagnosis', 'gate_source']) for id, patient_df in nindc_df.groupby('gate_source')}

In [148]:
print('RRMS cell-type abundances')
rrms_df.groupby('cluster').count()

RRMS cell-type abundances


,CD69,CXCR5,CCR2,CD57,CD28,IL.9,IL.4,CD27,IL.3,IL.13,...,CD25,CD4,CD3,IL.21,IL.17A,CXCR4,CCR4,CD14,diagnosis,gate_source
cluster,,,,,,,,,,,,,,,,,,,,,
0,1114,1114,1114,1114,1114,1114,1114,1114,1114,1114,...,1114,1114,1114,1114,1114,1114,1114,1114,1114,1114
1,4302,4302,4302,4302,4302,4302,4302,4302,4302,4302,...,4302,4302,4302,4302,4302,4302,4302,4302,4302,4302
3,147,147,147,147,147,147,147,147,147,147,...,147,147,147,147,147,147,147,147,147,147
8,2113,2113,2113,2113,2113,2113,2113,2113,2113,2113,...,2113,2113,2113,2113,2113,2113,2113,2113,2113,2113
10,401,401,401,401,401,401,401,401,401,401,...,401,401,401,401,401,401,401,401,401,401
11,216,216,216,216,216,216,216,216,216,216,...,216,216,216,216,216,216,216,216,216,216
16,171,171,171,171,171,171,171,171,171,171,...,171,171,171,171,171,171,171,171,171,171


In [149]:
print('NINDC cell-type abundances')
nindc_df.groupby('cluster').count()

NINDC cell-type abundances


,CD69,CXCR5,CCR2,CD57,CD28,IL.9,IL.4,CD27,IL.3,IL.13,...,CD25,CD4,CD3,IL.21,IL.17A,CXCR4,CCR4,CD14,diagnosis,gate_source
cluster,,,,,,,,,,,,,,,,,,,,,
0,1141,1141,1141,1141,1141,1141,1141,1141,1141,1141,...,1141,1141,1141,1141,1141,1141,1141,1141,1141,1141
1,4501,4501,4501,4501,4501,4501,4501,4501,4501,4501,...,4501,4501,4501,4501,4501,4501,4501,4501,4501,4501
3,219,219,219,219,219,219,219,219,219,219,...,219,219,219,219,219,219,219,219,219,219
8,1858,1858,1858,1858,1858,1858,1858,1858,1858,1858,...,1858,1858,1858,1858,1858,1858,1858,1858,1858,1858
10,379,379,379,379,379,379,379,379,379,379,...,379,379,379,379,379,379,379,379,379,379
11,202,202,202,202,202,202,202,202,202,202,...,202,202,202,202,202,202,202,202,202,202
16,125,125,125,125,125,125,125,125,125,125,...,125,125,125,125,125,125,125,125,125,125


In [164]:
importlib.reload(cellCnn.ms.utils.helpers)
from cellCnn.ms.utils.helpers import *

#### here we could see freq differences across the 2 groups
print('Frequencies: ')
rrms_patients_freq = {id: calc_frequencies(patient_df, cluster_to_celltype_dict, return_list=True) for id, patient_df in rrms_patients2df.items()}
nindc_patients_freq = {id: calc_frequencies(patient_df, cluster_to_celltype_dict, return_list=True) for id, patient_df in nindc_patients2df.items()}
print('DONE')
# we got 31 patients each

Frequencies: 
DONE


In [165]:
#### To get true frequencies we need to get rid of 0 entries (there are patient without some cells type due to data)
print('RRMS')
rrms_freq_df = pd.DataFrame(list(rrms_patients_freq.values()), columns=list(cluster_to_celltype_dict.values()))
rrms_freq_df = rrms_freq_df.replace(0, np.NaN) ## this lets us skip the 0 entries
rrms_freq_df.describe()

RRMS


,b,cd4,nkt,cd8,nk,my,dg
count,31.000000,31.000000,28.000000,31.000000,31.000000,31.000000,25.000000
mean,0.145822,0.470019,0.034702,0.242621,0.047500,0.038641,0.029826
std,0.062796,0.127876,0.043881,0.068406,0.031251,0.041190,0.026293
min,0.044118,0.222222,0.003953,0.120543,0.008264,0.004662,0.001832
25%,0.098750,0.393336,0.008248,0.193943,0.022284,0.017099,0.011655
50%,0.136364,0.461538,0.018236,0.243386,0.045455,0.023810,0.019802
75%,0.190358,0.543535,0.032926,0.283517,0.057529,0.044735,0.042857
max,0.285714,0.738540,0.178571,0.349057,0.125000,0.222222,0.128205


In [166]:
print('NINDC')
nindc_freq_df = pd.DataFrame(list(nindc_patients_freq.values()), columns=list(cluster_to_celltype_dict.values()))
nindc_freq_df = nindc_freq_df.replace(0, np.NaN)
nindc_freq_df.describe()

NINDC


,b,cd4,nkt,cd8,nk,my,dg
count,31.000000,31.000000,30.000000,31.000000,30.000000,29.000000,26.000000
mean,0.160800,0.440072,0.043986,0.245588,0.059403,0.039039,0.020230
std,0.107951,0.155058,0.038204,0.094339,0.047733,0.033544,0.020225
min,0.033149,0.189189,0.001524,0.046559,0.011236,0.001706,0.002841
25%,0.076150,0.329932,0.012810,0.206028,0.024907,0.014587,0.007189
50%,0.142857,0.390476,0.039086,0.247573,0.037789,0.031250,0.017101
75%,0.215665,0.516367,0.067342,0.290557,0.086959,0.050000,0.024923
max,0.433198,0.784572,0.142857,0.410061,0.200000,0.142857,0.106557


In [192]:
### desease states 1 = RRMS and 0 = NINDC
selection_pool_rrms_cd4, too_few_data_rrms_cd4 = get_chunks_from_df(rrms_patients2df, freq_df=rrms_patients_freq, desease_state=1, cluster=1, batch_size=40)
selection_pool_nindc_cd4, too_few_data_nindc_cd4 = get_chunks_from_df(nindc_patients2df, freq_df=nindc_patients_freq, desease_state=0, cluster=1, batch_size=40)
#todo make sure list are equally long:
if len(selection_pool_rrms_cd4)>len(selection_pool_nindc_cd4):
    selection_pool_rrms_cd4 = selection_pool_rrms_cd4[:len(selection_pool_nindc_cd4)]
elif len(selection_pool_rrms_cd4)<len(selection_pool_nindc_cd4):
    selection_pool_nindc_cd4 = selection_pool_nindc_cd4[:len(selection_pool_rrms_cd4)]

all_chunks = selection_pool_rrms_cd4 + selection_pool_nindc_cd4
np.random.shuffle(all_chunks)

In [193]:
X = [selection[0] for selection in all_chunks]
cd4 = [selection[1] for selection in all_chunks]
Y = [selection[2] for selection in all_chunks]
print('prep done')

prep done


In [194]:
X_test, X_train, X_valid, y_test, y_train, y_valid = split_test_valid_train(
    X=X,
    y=Y,
    test_perc=test_perc,
    train_perc=train_perc,
    valid_perc=0.5, seed=rand_seed)

In [195]:
outdir_pheno = 'ms_pheno_class_cd4_chunks'
model = get_fitted_model(X_train, X_valid, y_train, y_valid,
                         nrun=15, ncell=200, nsubset=batch_size_pheno,
                         nfilters=[3, 15, 25, 35], coeff_l1=0,
                         coeff_l2=1e-4, max_epochs=100, learning_rate=0.005,
                         per_sample=False, regression=False,
                         outdir=outdir_pheno)
#### NOTE:
#### it performs Strat.KFold anyways (even if i dont manually divide them into train valids... )
#### bad thing is that i dont have any test yet
#### made Errors:
#### 1. I put in y_unique as the true class labels (no integer in a range) and got a list indes out of bound
print('classification task DONE')





Epoch 1/100
8/9 [=========================>....] - ETA: 0s - loss: 0.6622 - accuracy: 0.6837
Epoch 00001: val_loss improved from inf to 0.66036, saving model to ms_pheno_class_cd4_chunks/nnet_run_0.hdf5
9/9 [==============================] - 1s 92ms/step - loss: 0.6605 - accuracy: 0.6887 - val_loss: 0.6604 - val_accuracy: 0.6964
Epoch 2/100
8/9 [=========================>....] - ETA: 0s - loss: 0.5125 - accuracy: 0.8806

In [196]:
results = model.results
scaler = results['scaler']
filters = results['w_best_net']
save_results(results, outdir, cytokines)

In [197]:
# # Model predictions: to beat
#  [[0.06793241 0.93206759]
#  [0.95944599 0.04055399]
#  [0.05254236 0.94745767]
#  [0.02734859 0.9726514 ]]
test_pred = model.predict(X_test)
#print_regression_model_stats(test_pred, y_test)
print(f'ROC AUC : {roc_auc_score(y_test, test_pred[:,1])}')
print(f'Pred: {test_pred}')
print(f'Obs: {y_test}')
# This is classification. ..
# calculate area under the ROC curve for the test set


ROC AUC : 0.5
Pred: [[0.55793891 0.4420611 ]
 [0.49376318 0.50623677]
 [0.39560146 0.60439855]
 [0.44893676 0.55106324]
 [0.60760883 0.39239118]
 [0.67153438 0.32846562]
 [0.45740779 0.54259223]]
Obs: [0, 0, 1, 1, 1, 1, 0]


In [131]:
# plot the results of the CellCnn analysis for the test samples in the output directory
_ = plot_results(model.results, X_test, y_test,
                 cytokines, outdir + '/plots', filter_response_thres=0,
                 filter_diff_thres=0.2, group_a='RRMS', group_b='NINDC')
print('done')
### plotting stuff

done


In [3]:
#####################################################################
#####################################################################
################### First try of cd4 T cell freq. ###################
#####################################################################
#####################################################################

In [12]:
celltype_dfs = dict()
grouped_df = df.groupby('cluster')
for cluster in cluster_to_celltype_dict.keys():
    celltype_dfs[cluster] = [grouped_df.get_group(cluster)]

for val in celltype_dfs.values():  ### crosscheck with my ms_data set
    print(len(val[0]))
### since my date for CD4 T cells is biggest i start with group 1

2255
8803
366
3971
780
296
418


In [13]:
selection = 1
cd4_df = celltype_dfs[selection][0]

In [16]:
cd4_idx_chunks = get_chunks(cd4_df.index, batch_size_cd4)

In [19]:
counter_vals = []
for key, val in celltype_dfs.items():
    if key != selection:
        counter_vals.append(val[0])

counter_df = pd.concat(counter_vals)
counter_idx_chunks = get_chunks(counter_df.index, batch_size_cd4)
len(counter_idx_chunks)

15

In [18]:
freq = [whole_freq_dict[selection] for i in range(len(cd4_idx_chunks))] + [0 for i in range(len(counter_idx_chunks))]
df_reduced = df.drop(columns=['diagnosis', 'cluster'])
cd4_X = [df_reduced.loc[cd4_idx, :] for cd4_idx in cd4_idx_chunks]
chunks = [df_reduced.loc[counter_idx, :] for counter_idx in counter_idx_chunks]
cd4_X = cd4_X + chunks
cd4_X

NameError: name 'counter_idx_chunks' is not defined

In [36]:
X_test_cd4, X_train_cd4, X_valid_cd4, y_test_cd4, y_train_cd4, y_valid_cd4 = split_test_valid_train(X=cd4_X, y=freq,
                                                                                                    test_perc=test_perc,
                                                                                                    train_perc=train_perc,
                                                                                                    valid_perc=0.5,
                                                                                                    seed=rand_seed)

In [118]:
outdir = 'out_ms_cd4_2'
model_cd4 = get_fitted_model(X_train=X_train_cd4, X_valid=X_valid_cd4,
                             y_train=y_train_cd4, y_valid=y_valid_cd4,
                             nrun=15, ncell=200, nsubset=batch_size_cd4,
                             nfilters=[3, 15, 36], coeff_l2=1e-5, coeff_l1=None,
                             max_epochs=50, outdir=outdir,
                             regression=True, per_sample=True)

Epoch 1/50
61/61 [==============================] - 5s 79ms/step - loss: nan - val_loss: nan

An exception was raised during training the network.
Unable to open file (unable to open file: name = 'out_ms_cd4_2/nnet_run_0.hdf5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)


Epoch 1/50
61/61 [==============================] - 5s 86ms/step - loss: nan - val_loss: nan

An exception was raised during training the network.
Unable to open file (unable to open file: name = 'out_ms_cd4_2/nnet_run_1.hdf5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)


Epoch 1/50
61/61 [==============================] - 4s 69ms/step - loss: nan - val_loss: nan

...

Epoch 1/50
61/61 [==============================] - 4s 71ms/step - loss: nan - val_loss: nan

An exception was raised during training the network.
Unable to open file (unable to open file: name = 'out_ms_cd4_2/nnet_run_13.hdf5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)


Epoch 1/50
61/61 [==============================] - 4s 72ms/step - loss: nan - val_loss: nan

An exception was raised during training the network.
Unable to open file (unable to open file: name = 'out_ms_cd4_2/nnet_run_14.hdf5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)


KeyError: 14

In [119]:
results = model_cd4.results
scaler = results['scaler']
filters = results['w_best_net']
save_results(results, outdir, cytokines)

NameError: name 'model_cd4' is not defined

In [120]:
# # Model predictions: to beat
#model = CellCnn(nrun=15, ncell=200, nsubset=batch_size_cd4, nfilter_choice=[35],
#                coeff_l2=1e-5, max_epochs=50, per_sample=Tru
#  [[0.5068497 ]
#  [0.51000313]
#  [0.07073919]
#  [0.02683752]
#  [0.51068987]]
test_pred_cd4 = model_cd4.predict(X_test_cd4)
print_regression_model_stats(test_pred_cd4, y_test_cd4)

NameError: name 'model_cd4' is not defined

In [68]:
# plot the results of the CellCnn analysis for the test samples in the output directory
_ = plot_results(model_cd4.results, X_test_cd4, y_test_cd4,
                 cytokines, outdir + '/plots', filter_response_thres=0,
                 filter_diff_thres=0.2, group_a='RRMS', group_b='NINDC')
print('done')

KeyError: 1

In [ ]:
#####################################################################
#####################################################################
################### First try of MTL (pheno + cd4) T cell freq. ###################
#####################################################################
#####################################################################
# from: https://machinelearningmastery.com/neural-network-models-for-combined-classification-and-regression/#:~:text=Regression%20refers%20to%20predictive%20modeling,labels%20for%20a%20given%20input.

In [20]:
cd4_df.head() ## from above
cd4_df_rrms = cd4_df[cd4_df['diagnosis'] == 'RRMS']
cd4_df_nindc = cd4_df[cd4_df['diagnosis'] == 'NINDC']
cd4_df_rrms_chunks_idx = get_chunks(cd4_df_rrms.index, batch_size_cd4)
cd4_df_nindc_chunks_idx = get_chunks(cd4_df_nindc.index, batch_size_cd4)

In [21]:
counter_df.head() ## from above
counter_df_rrms = counter_df[counter_df['diagnosis'] == 'RRMS']
counter_df_nindc = counter_df[counter_df['diagnosis'] == 'NINDC']
counter_df_rrms_chunks_idx = get_chunks(counter_df_rrms.index, batch_size_cd4)
counter_df_nindc_chunks_idx = get_chunks(counter_df_nindc.index, batch_size_cd4)

In [25]:
freq_pheno_rrms = [(whole_freq_dict[selection], 1) for i in range(len(cd4_df_rrms_chunks_idx))] + [(0, 1) for i in range(len(counter_df_rrms_chunks_idx))]
freq_pheno_nindc = [(whole_freq_dict[selection], 0) for i in range(len(cd4_df_nindc_chunks_idx))] + [(0, 0) for i in range(len(counter_df_nindc_chunks_idx))]
df_reduced = df.drop(columns=['diagnosis', 'cluster'])
cd4_rrms_X = [df_reduced.loc[cd4_idx, :] for cd4_idx in cd4_df_rrms_chunks_idx]
cd4_nindc_X = [df_reduced.loc[cd4_idx, :] for cd4_idx in cd4_df_nindc_chunks_idx]
chunks_rrms = [df_reduced.loc[counter_idx, :] for counter_idx in counter_df_rrms_chunks_idx]
chunks_nindc = [df_reduced.loc[counter_idx, :] for counter_idx in counter_df_nindc_chunks_idx]
cd4_mtl_X = cd4_rrms_X + chunks_rrms + cd4_nindc_X + chunks_nindc
cd4_pheno_mtl_y = freq_pheno_rrms + freq_pheno_nindc

In [28]:
X_test_mtl_1, X_train_mtl_1, X_valid_mtl_1, y_test_mtl_1, y_train_mtl_1, y_valid_mtl_1 = split_test_valid_train(X=cd4_mtl_X, y=cd4_pheno_mtl_y,
                                                                                                    test_perc=test_perc,
                                                                                                    train_perc=train_perc,
                                                                                                    valid_perc=0.5,
                                                                                                    seed=rand_seed)


In [33]:
[item[0] for item in y_test_mtl_1]

[0.5212268340339866, 0, 0.5212268340339866, 0, 0.5212268340339866]

In [34]:
[item[1] for item in y_test_mtl_1]

[0, 1, 1, 0, 0]

In [69]:
importlib.reload(cellCnn.ms.utils.helpers)
importlib.reload(cellCnn.model)
from cellCnn.ms.utils.helpers import *
from cellCnn.model import CellCnn


outdir = 'out_ms_mtl_cd4_pheno'
model_cd4 = get_fitted_model(X_train=X_train_mtl_1, X_valid=X_valid_mtl_1,
                             y_train=[[item[0] for item in y_train_mtl_1], [item[1] for item in y_train_mtl_1]],
                             y_valid=[[item[0] for item in y_valid_mtl_1], [item[1] for item in y_valid_mtl_1]],
                             nrun=15, ncell=200, nsubset=batch_size_cd4,
                             nfilters=[3, 15, 36], coeff_l2=1e-5, coeff_l1=None,
                             max_epochs=50, outdir=outdir,
                             regression=True, per_sample=True)


An exception was raised during training the network.
Data cardinality is ambiguous:
  x sizes: 12650, 1090
  y sizes: 12650, 1090
Please provide data which shares the same first dimension.
An exception was raised during training the network.
Data cardinality is ambiguous:
  x sizes: 12650, 1090
  y sizes: 12650, 1090
Please provide data which shares the same first dimension.
